In [2]:
import math
import numpy as np
import matplotlib as plt
import turtle

In [3]:
def init_positions(n, x_lower, x_upper, y_lower, y_upper):
    #Dictionary consisting of numpy arrays corresponding to x-y coordinates
    #of each agent
    
    positions = {}
    
    for i in range(n):
        
        x_coordinate = random.randrange(x_lower, x_upper)
        y_coordinate = random.randrange(y_lower, y_upper)
        
        positions[i] = np.asarray([x_coordinate, y_coordinate])
        
    return positions
    
def calculate_interagent_distance(n, positions):
    # Given : n x 2 matrix
    # return : n x n matrix
    
    interagent_distances = []
    for i in range(n):
        
        temporary_array = []
        for j in range(n):
            distance = np.linalg.norm(positions[i]-positions[j])
            temporary_array.append(distance)
            
        interagent_distances.append(temporary_array)
        
    return interagent_distances
            
    

def calculate_interagent_theta(n, positions):
    # Given : n x 2 matrix
    # return : n x n matrix
    interagent_angles = []
    for i in range(n):
        temporary_array = []
        for j in range(n):
            
            cos_theta = np.dot(positions[i],positions[j])/(np.linalg.norm(positions[i])*np.linalg.norm(positions[j]))
            angle = math.degrees(math.acos(cos_theta))
            temporary_array.append(angle)
            
        interagent_angles.append(temporary_array)
    
    return interagent_angles

def calculate_force(distance, Fmax, R, G):
    if distance > 1.5*R:
        return 0.0
    
    else:
        F = G/(distance**2)
        if F > Fmax:
            F = Fmax
        
        if distance < R:
            F = -F
        
        return F
    
def update_velocity(fx, fy, dt):
    vx = dt*fx
    vy = dt*fy
    
    return vx, vy

def update_position(vx, vy, dt):
    x = dt*vx
    y = dt*vy
    
    return x,y

def distance_to_move(dx, dy):
    return math.floor(math.sqrt(dx**2+dy**2))
    
def bearing_angle(dx, dy):
    return math.degrees(math.atan2(dy,dx))

    

In [ ]:
# initialise positions for all agents and agent velocities to zero and simulate
# calculate the distance, bearing of all robot for each robot - r,theta
# for each timestep:
    # For each robot do the following at each timestep:
        # for each neighbouring agent do the following:
            # calculate forces due to each robot
            # break into x,y component and calculate cummulative
        # calculate the updated velocity 
        # calculate new position of robot
        # calculate, turn and distance to move for each agent
    # simulate movement and set velocity back to zero(friction)
